In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import glob
import matplotlib.pyplot as plt
import sklearn
from collections import Counter
import glob
import pickle
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD, Adam
from random import shuffle
%matplotlib inline

/Users/harryramli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/harryramli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/harryramli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/harryramli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/harryramli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/harryramli/an

In [122]:
def create_model_convs(input_shape):
    model = Sequential()

    model.add(Conv2D(32,(3,3), padding= 'same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3), padding= 'valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64,(3,3), padding= 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3), padding= 'valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256,(3,3), padding= 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(256,(3,3), padding= 'valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model 

In [32]:
#to check which file has an issue 
for path in glob.glob('../Music test rprod/spectrogram_final/*'):
    print(path.split('/')[3])
    crop = cv2.imread(path)[:,:200]

348.png
412.png
374.png
360.png
406.png
638.png
176.png
88.png
610.png
604.png
162.png
189.png
837.png
77.png
823.png
63.png
980.png
758.png
994.png
764.png
770.png
943.png
957.png
228.png
566.png
200.png
214.png
572.png
599.png
598.png
215.png
573.png
567.png
201.png
229.png
956.png
942.png
771.png
765.png
995.png
759.png
981.png
62.png
822.png
188.png
76.png
836.png
605.png
163.png
177.png
611.png
89.png
639.png
361.png
407.png
413.png
375.png
349.png
439.png
405.png
363.png
377.png
411.png
388.png
149.png
161.png
607.png
613.png
175.png
820.png
60.png
834.png
74.png
808.png
48.png
997.png
983.png
773.png
767.png
954.png
798.png
940.png
968.png
559.png
571.png
217.png
203.png
565.png
202.png
564.png
570.png
216.png
558.png
969.png
941.png
799.png
955.png
766.png
772.png
982.png
996.png
49.png
809.png
75.png
835.png
61.png
821.png
612.png
174.png
160.png
606.png
148.png
389.png
376.png
410.png
404.png
362.png
438.png
366.png
400.png
414.png
372.png
428.png
399.png
602.png
164.png
170.

TypeError: 'NoneType' object is not subscriptable

In [37]:
for path in glob.glob('../Music test rprod/spectrogram_final/*'):
    
    #file 694.png has an issue, so skipping it
    if(path.split('/')[3]=='694.png'):
        continue
    img = cv2.imread(path)
    
    #cropping to get 0-30s and 30-60s spectrograms from
    crop1 = img[:,:200] 
    crop2 = img[:,200:400] 
    cv2.imwrite('../Music test rprod/0-30s/'+path.split('/')[3],crop1)
    cv2.imwrite('../Music test rprod/30-60s/'+path.split('/')[3],crop2)

In [102]:
map_genre = {
    0:'Pop',
    1:'Rock',
    2:'Jazz'
}

reverse_genre = {
    'Pop':0,
    'Rock':1,
    'Jazz':2
}

batch_size = 100
epochs = 5
num_classes = len(map_genre)
input_shape = (200,362,3)


In [133]:
X0_train, X0_test, y0_train, y0_test = [],[],[],[]
X30_train, X30_test, y30_train, y30_test = [],[],[],[]
data0_data, data0_target = [],[]
data30_data, data30_target = [],[]

annotation = pd.read_csv('annotation.csv')
label = annotation['genre 1']
label = label.replace(reverse_genre)
pic_number = annotation['file_audio']

#inserting a column in the annotation.csv file indicating the equivalent labelling
annotation['index_code'] = label 

In [134]:
for path in glob.glob('../Music test rprod/0-30s/*'):
    picnum = path.split('/')[3].split('.')[0]
    picindex = int(picnum)
    #obtaining label according to the image number and its corresponding label value in the annotation file
    temp1 = annotation[annotation['file_audio']==picindex]['index_code'].iloc[0]
    data0_target.append(temp1)
    temp1 = cv2.imread(path)
    temp1 = cv2.resize(temp1, (200,362)).astype('float32')
    data0_data.append(temp1)
    
for path in glob.glob('../Music test rprod/30-60s/*'):
    picnum = path.split('/')[3].split('.')[0]
    picindex = int(picnum)
    #obtaining label according to the image number and its corresponding label value in the annotation file
    temp2 = annotation[annotation['file_audio']==picindex]['index_code'].iloc[0]
    data30_target.append(temp2)
    temp2 = cv2.imread(path)
    temp2 = cv2.resize(temp2, (200,362)).astype('float32')
    data30_data.append(temp2)

In [135]:
from sklearn.model_selection import train_test_split

#splitting data to training and testing data
X0_train,X0_test,y0_train,y0_test = train_test_split(data0_data,data0_target,test_size=0.33,random_state=40)
X30_train,X30_test,y30_train,y30_test = train_test_split(data30_data,data30_target, test_size=0.33, random_state=40)

In [137]:
#list to numpy array to fit into keras model
X0_train = np.array(X0_train)
X0_test = np.array(X0_test)
y0_train = keras.utils.to_categorical(y0_train,num_classes=num_classes)
y0_test = keras.utils.to_categorical(y0_test,num_classes=num_classes)

X30_train = np.array(X30_train)
X30_test = np.array(X30_test)
y30_train = keras.utils.to_categorical(y30_train,num_classes=num_classes)
y30_test = keras.utils.to_categorical(y30_test,num_classes=num_classes)

In [ ]:
#creating convolutional neural network to test accuracy of model trained using 0-30s of spectrograms
model = create_model_convs(input_shape=(362,200,3))
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
model.fit(X0_train,y0_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X0_test,y0_test))

#to be trained on a faster computer / server

Train on 668 samples, validate on 330 samples
Epoch 1/5


In [ ]:
#creating convolutional neural network to test accuracy of model trained using 30-60s of spectrograms
model = create_model_convs(input_shape=(362,200,3))
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
model.fit(X30_train,y30_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X30_test,y30_test))

#to be trained on a faster computer / server